# VacationPy
---

In [45]:
# load dependencies
import hvplot.pandas
from bokeh.resources import INLINE
import pandas as pd
import requests
import os

# Import API key
from api_keys import geoapify_key

In [46]:
# read in the data generated by `WeatherPy.ipynb`
file_path = os.path.join('output_data', 'city_weather.csv')

city_data_df = pd.read_csv(file_path)[['City', 'Country', 'Latitude', 'Longitude', 'Temperature (Max)', 'Humidity', 'Cloudiness', 'Wind Speed']]
city_data_df.head()

,City,Country,Latitude,Longitude,Temperature (Max),Humidity,Cloudiness,Wind Speed
0,Palmer,US,61.599570,-149.111090,276.10,84,100,1.54
1,Tuktoyaktuk,CA,69.444019,-133.031904,250.15,77,40,6.17
2,Ushuaia,AR,-54.806116,-68.318497,278.98,70,75,6.69
3,Thule,GL,77.468636,-69.222275,255.25,76,1,2.14
4,Zalantun City,CN,47.746019,121.970532,284.04,19,4,4.44


### Map All Cities in the List with Markers Scaled by Humidity

In [47]:
# configure a map to show all cities in the dataframe with the size of the markers indicating humidity
map_plot = city_data_df.hvplot.points(
    'Longitude',
    'Latitude',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 0.8,
    color = 'City',
    hover_cols = 'all'
)

# save and display the map plot
hvplot.save(map_plot, 'output_data/all_cities_mapped_humidity_size.html', resources=INLINE)
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,index,Country,Temperature (Max),Cloudiness,Wind Speed)

### Find Ideal Climate Cities and Map w/ Nearest Hotel

#### Create a Narrowed-Down DataFrame by Weather

In [48]:
# limit `city_data_df` to ideal weather conditions and store relevant information
hotel_df = city_data_df.loc[(city_data_df['Temperature (Max)'] - 273.15 > 22) & ((city_data_df['Cloudiness'] == 0))].reset_index(drop=True)
hotel_df = hotel_df[['City', 'Country', 'Latitude', 'Longitude', 'Humidity']]
hotel_df

,City,Country,Latitude,Longitude,Humidity
0,New York County,US,40.712728,-74.006015,44
1,Biloela,AU,-24.399771,150.514187,18
2,Del Rio,US,29.365541,-100.894698,44
3,Acapulco,MX,16.868050,-99.894018,74
4,Lalmohan,BD,22.341058,90.733433,44
5,Srikakulam,IN,18.294931,83.893884,31
6,Paraul,NP,26.761999,85.820527,9
7,Puri,IN,19.807608,85.825254,47
8,Lashio Township,MM,22.933333,97.750000,11
9,Hervey Bay,AU,-25.298578,152.853522,35


#### Retrieve Hotel Data from `Geoapify`

In [49]:
# instantiate a url string for formatting to make the Geoapify API call
hoteldata_url = 'https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:{lng},{lat},10000&bias=proximity:{lng},{lat}&limit=20&apiKey={API_key}'

# iterate through `hotel_df`
for index, row in hotel_df.iterrows():

    # make the API call and pick out the first hotel in the response, or note that no hotel was found
    try:
        response = requests.get(hoteldata_url.format(lng=row['Longitude'], lat=row['Latitude'], API_key=geoapify_key))
        data = response.json()
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]

    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# display sample data
hotel_df

New York County - nearest hotel: The Beekman, A Thompson Hotel
Biloela - nearest hotel: No hotel found
Del Rio - nearest hotel: La Quinta Inn by Wyndham Del Rio
Acapulco - nearest hotel: Hotel del Valle
Lalmohan - nearest hotel: No hotel found
Srikakulam - nearest hotel: Venus Vegetarian Meals
Paraul - nearest hotel: Devindra Meat House
Puri - nearest hotel: Marchikote Inn
Lashio Township - nearest hotel: Golden Kaniri Hotel
Hervey Bay - nearest hotel: Hervey Bay Hotel
Maryborough - nearest hotel: Post Office Hotel
Narendranagar - nearest hotel: Ananda in the Himalayas
Makakilo City - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
Fukue - nearest hotel: 倉敷由加温泉ホテル 山桃花
Sharjah - nearest hotel: Monaco Hotel


,City,Country,Latitude,Longitude,Humidity,Hotel Name
0,New York County,US,40.712728,-74.006015,44,"The Beekman, A Thompson Hotel"
1,Biloela,AU,-24.399771,150.514187,18,No hotel found
2,Del Rio,US,29.365541,-100.894698,44,La Quinta Inn by Wyndham Del Rio
3,Acapulco,MX,16.868050,-99.894018,74,Hotel del Valle
4,Lalmohan,BD,22.341058,90.733433,44,No hotel found
5,Srikakulam,IN,18.294931,83.893884,31,Venus Vegetarian Meals
6,Paraul,NP,26.761999,85.820527,9,Devindra Meat House
7,Puri,IN,19.807608,85.825254,47,Marchikote Inn
8,Lashio Township,MM,22.933333,97.750000,11,Golden Kaniri Hotel
9,Hervey Bay,AU,-25.298578,152.853522,35,Hervey Bay Hotel


#### Map the Results for Ideal Weather Locations with Nearest Hotel

In [50]:
# configure a map to show cities in the refined dataframe with the size of the markers indicating humidity
map_plot = hotel_df.hvplot.points(
    'Longitude',
    'Latitude',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 2,
    color = 'City',
    hover_cols = 'all'
)

# save and display the map plot
hvplot.save(map_plot, 'output_data/ideal_weather_hotel_map_humidity_size.html', resources=INLINE)
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,index,Country,Hotel Name)